In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import wandb
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from sklearn.model_selection import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
device = 'cuda'
PROJECT_NAME = 'Spam-Clf'

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('$100')

['$', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['how','hi'])

array([0., 1.], dtype=float32)

In [8]:
data = pd.read_csv('./Language Detection.csv').dropna()[:6250]

In [9]:
X = data['Text']
y = data['Language']

In [10]:
all_words = []
all_data = []
tags = []

In [11]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

6250it [00:02, 2569.97it/s]


In [12]:
np.random.shuffle(all_words)
np.random.shuffle(tags)

In [13]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [14]:
np.random.shuffle(all_words)
np.random.shuffle(tags)

In [15]:
X = []
y = []

In [16]:
for sentence,tag in tqdm(all_data):
    X.append(bag_of_words(sentence,all_words))
    y.append(tags.index(tag))

100%|██████████████████████████████████████| 6250/6250 [00:37<00:00, 167.67it/s]


In [17]:
X[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [18]:
y[0]

4

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)

In [20]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).long()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).long()

In [39]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [40]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)
    return acc

In [59]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.activation = ReLU()
        self.iters = 12
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.batchnorm = BatchNorm1d(hidden)
        self.output = Linear(hidden,len(tags))
    
    def forward(self,X):
        preds = self.batchnorm(self.activation(self.linear1(X)))
        for _ in range(self.iters):
            preds = self.batchnorm(self.activation(self.linear2(preds)))
        preds = self.output(preds)
        return preds

In [60]:
model = Model(1024).to(device)

In [61]:
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [62]:
epochs = 100

In [63]:
batch_size = 32

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline-batchnorm-add')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

 23%|█████████▋                                | 23/100 [00:59<03:17,  2.57s/it]

In [ ]:
batchnorm